In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
REPO_NAME = 'sewer-nfl'
import sys, os
cwd = str(os.getcwd())
repo_dir = cwd[:cwd.find(REPO_NAME)+len(REPO_NAME)]
sys.path.insert(0,repo_dir)

In [3]:
import pandas as pd
import nfl_data_py as nflreadr

In [4]:
from warehouse.utilities.elo import calculate_elo_metric

In [5]:
api_data = nflreadr.import_pbp_data(range(2021,2023))

2021 done.
2022 done.
Downcasting floats.


In [6]:
api_data = api_data[api_data['week']<=18]

In [7]:
from warehouse.pipelines.pbp.pbp import player_involvement, player_involved, players_involved

In [8]:
player_involvement_df = player_involvement(pbp_data=api_data)

In [9]:
player_involved(player_involvement_df=player_involvement_df,
                #play_id = 40.0,
                #game_id = '2021_01_ARI_TEN',
                player_id = '00-0032560')

,play_id,game_id,home_team,away_team,posteam,posteam_type,variable,value
1,40.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032560
17,451.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032560
27,701.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032560
35,885.0,2021_01_ARI_TEN,TEN,ARI,ARI,away,o1,00-0032560
56,1385.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032560
...,...,...,...,...,...,...,...,...
1248560,298.0,2022_18_ARI_SF,SF,ARI,SF,home,d2,00-0032560
1248629,1959.0,2022_18_ARI_SF,SF,ARI,SF,home,d2,00-0032560
1248641,2211.0,2022_18_ARI_SF,SF,ARI,SF,home,d2,00-0032560
1248654,2508.0,2022_18_ARI_SF,SF,ARI,SF,home,d2,00-0032560


In [40]:
meta_columns = [
    'game_id',
    'old_game_id',
    'week',
    'home_team',
    'away_team',
    'spread_line'
]

play_columns = [
    'play_id',
    'play_type',
    'desc',
    'yards_gained',
    'passer_player_id',
    'receiver_player_id',
    'rusher_player_id',
    'offense_players',
    'defense_players'
]

epa_columns = [
    'epa',
    'air_epa',
    'yac_epa'
]

In [18]:
# Player Id Lookup function
rosters = nflreadr.import_rosters(years=[2021,2022])

In [21]:
r_columns = [
    'season',
    'week',
    'team',
    'position',
    'player_name',
    'player_id'
]
rosters[r_columns]

,season,week,team,position,player_name,player_id
0,2021,19,ARI,DB,Chris Banjo,00-0029747
1,2021,19,ARI,DB,Robert Alford,00-0030528
2,2021,19,ARI,DB,Bashaud Breeland,00-0031052
3,2021,19,ARI,DB,Malcolm Butler,00-0031218
4,2021,19,ARI,DB,Charles Washington,00-0032560
...,...,...,...,...,...,...
2636,2022,18,WAS,WR,Antonio Gandy-Golden,00-0036340
2637,2022,18,WAS,WR,Dyami Brown,00-0036626
2638,2022,18,WAS,WR,Dax Milne,00-0036671
2639,2022,18,WAS,WR,Jahan Dotson,00-0037741


In [35]:
# Build ELO Dataset

epa_dataset = api_data[meta_columns + play_columns + epa_columns]

In [42]:
player_involvement_df

,play_id,game_id,home_team,away_team,posteam,posteam_type,variable,value
0,1.0,2021_01_ARI_TEN,TEN,ARI,None,None,o1,
1,40.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032560
2,55.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0029413
3,76.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0029413
4,100.0,2021_01_ARI_TEN,TEN,ARI,TEN,home,o1,00-0032355
...,...,...,...,...,...,...,...,...
2117605,3677.0,2022_18_TEN_JAX,JAX,TEN,TEN,away,d11,00-0033941
2117606,3708.0,2022_18_TEN_JAX,JAX,TEN,JAX,home,d11,00-0033782
2117607,3740.0,2022_18_TEN_JAX,JAX,TEN,None,None,d11,None
2117608,3729.0,2022_18_TEN_JAX,JAX,TEN,JAX,home,d11,00-0033782
